In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
!pip install folium

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


from bs4 import BeautifulSoup
import requests

     |████████████████████████████████| 94 kB 5.6 MB/s  eta 0:00:01
Libraries imported.


In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago'
df = pd.read_html(url)
df[0].head()

,Neighborhood,Community area
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Arcadia Terrace,West Ridge
4,Archer Heights,Archer Heights


In [4]:
df[0].shape

(247, 2)

In [5]:
df[0]

,Neighborhood,Community area
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Arcadia Terrace,West Ridge
4,Archer Heights,Archer Heights
5,Armour Square,Armour Square
6,Ashburn,Ashburn
7,Ashburn Estates,Ashburn
8,Auburn Gresham,Auburn Gresham
9,Avalon Park,Avalon Park


In [25]:
# Define Foursquare Credentials and Version

CLIENT_ID = 'S2QYOB0NIQGNWAYWPHSWB0Y1H5EBR1AC5C01QPVFUZIXHCE0' # your Foursquare ID
CLIENT_SECRET = 'UENP1UOYQ2MGJKGCJISP0HHCM5VTCUXPYS1VWSRLWHDDZKLP' # your Foursquare Secret
VERSION = '20210909' # Foursquare API version

In [7]:
def geo_location(address):
    # get geo location of address
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude


def get_venues(lat,lng):
    #set variables
    radius=400
    LIMIT=100
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


def get_venue_details(venue_id):
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    # get all the data
    results = requests.get(url).json()
    print(results)
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


def get_chicago_data():
    url='https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    # define the dataframe columns
    column_names = ['Neighborhood', 'Community area', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    chicago_data = pd.DataFrame(columns=column_names)
    for data in features:
        neighborhood = data['properties']['Neighborhood'] 
        community_area = data['properties']['Community area']
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
        chicago_data = chicago_data.append({'Neighborhood': neighborhood,
                                          'Community area': community_area,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    return chicago_data

In [8]:
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_1b66257f0c58427797a4f45e8207c431 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='mnXZ3Stovzpo1TP4cmgbAnquFPn-biSiXc4I6PLjIjEK',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_1b66257f0c58427797a4f45e8207c431.get_object(Bucket='capstoneprojectchicagooverview-donotdelete-pr-h5zuqhbtjxdm4x',Key='Chicago_Community_Area_Lat_Long.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_chi = pd.read_csv(body)
df_chi.head()

,Number,Community Area,Latitude,Longitude
0,8,Near North Side,41.9039,87.6315
1,32,Loop,41.8786,87.6251
2,33,Near South Side,41.8608,87.6257
3,5,North Center,41.9467,87.6883
4,6,Lake View,41.9398,87.6589


In [9]:
df_chi

,Number,Community Area,Latitude,Longitude
0,8,Near North Side,41.9039,87.6315
1,32,Loop,41.8786,87.6251
2,33,Near South Side,41.8608,87.6257
3,5,North Center,41.9467,87.6883
4,6,Lake View,41.9398,87.6589
5,7,Lincoln Park,41.9255,87.6488
6,21,Avondale,41.9415,87.7025
7,22,Logan Square,41.9231,87.7093
8,1,Rogers Park,42.0126,87.6746
9,2,West Ridge,42.0006,87.6926


In [11]:
df_new = []
df_new = df[0].join(df_chi.set_index('Community Area'), on='Community area')
chi_data = df_new[['Number', 'Community area', 'Neighborhood', 'Latitude', 'Longitude']]
chi_data['Longitude']= chi_data['Longitude'] * -1
chi_data.head()

,Number,Community area,Neighborhood,Latitude,Longitude
0,14.0,Albany Park,Albany Park,41.9683,-87.7280
1,54.0,Riverdale,Altgeld Gardens,41.6611,-87.6038
2,77.0,Edgewater,Andersonville,41.9837,-87.6601
3,2.0,West Ridge,Arcadia Terrace,42.0006,-87.6926
4,57.0,Archer Heights,Archer Heights,41.8079,-87.7236


In [12]:
chi_data

,Number,Community area,Neighborhood,Latitude,Longitude
0,14.0,Albany Park,Albany Park,41.9683,-87.7280
1,54.0,Riverdale,Altgeld Gardens,41.6611,-87.6038
2,77.0,Edgewater,Andersonville,41.9837,-87.6601
3,2.0,West Ridge,Arcadia Terrace,42.0006,-87.6926
4,57.0,Archer Heights,Archer Heights,41.8079,-87.7236
5,34.0,Armour Square,Armour Square,41.8408,-87.6340
6,70.0,Ashburn,Ashburn,41.7497,-87.7120
7,70.0,Ashburn,Ashburn Estates,41.7497,-87.7120
8,71.0,Auburn Gresham,Auburn Gresham,41.7434,-87.6562
9,45.0,Avalon Park,Avalon Park,41.7442,-87.5856


In [13]:
address = 'Chicago, IL'

geolocator = Nominatim(user_agent="chi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Chicago are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Chicago are 41.8755616, -87.6244212.


In [20]:
chi_data = chi_data.dropna()

print (chi_data)

     Number          Community area                         Neighborhood  \
0      14.0             Albany Park                          Albany Park   
1      54.0               Riverdale                      Altgeld Gardens   
2      77.0               Edgewater                        Andersonville   
3       2.0              West Ridge                      Arcadia Terrace   
4      57.0          Archer Heights                       Archer Heights   
5      34.0           Armour Square                        Armour Square   
6      70.0                 Ashburn                              Ashburn   
7      70.0                 Ashburn                      Ashburn Estates   
8      71.0          Auburn Gresham                       Auburn Gresham   
9      45.0             Avalon Park                          Avalon Park   
10     21.0                Avondale                             Avondale   
11     16.0             Irving Park                     Avondale Gardens   
12     61.0 

In [21]:
# create map of Chicago using latitude and longitude values
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, community, neighborhood in zip(chi_data['Latitude'], chi_data['Longitude'], chi_data['Community area'], chi_data['Neighborhood']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

ValueError: too many values to unpack (expected 3)